# Estaciones de Calidad del Aire

## Configuracion del notebook <a id="config"></a>

Cargamos todos los paquetes necesarios para ejecutar el notebook

In [1]:
setwd("~/Repositories/AirQualityCOVID/")

In [2]:
# Loading
suppressMessages(library(saqgetr))
suppressMessages(library(tidyverse))
suppressMessages(library(lubridate))
suppressMessages(library(openxlsx))

## Datos geograficos de las estaciones <a id="estacionGeo"></a>

Seleccionamos aquellas estaciones que se encuentren en un núcleo con una
población mayor o igual a cien mil habitantes `poblacion >= 100 000 hab`.

En la hoja 8 `ciudades-100000-A-JA` de la base de datos de poblaciones (`estaciones-CA.xlsx`) aparecen todas las estaciones de trafico de las ciudades de mas de 100000 habitantes con sus nombres.

In [3]:
file <- "data/xlsx/estaciones-CA-JA.xlsx"
sheets <- c("todas", "traffic", "traffic-urban", "traffic-urban-2020",
            "traffic-suburban", "traffic-suburban-2020",
            "ciudades-100000", "ciudades-100000-A")
# xlsx files
sites.100mil <- read.xlsx(file, sheet=sheets[8])

#### Datos de las estaciones de `saqgetr` <a id="estacionesSaqgetr"></a>

Importamos la informacion de las estaciones de calidad de aire de españa obtenidas de la base de datos y filtramos segun los criterios de estudio.

| Variable | Valores |
|-------|-----------|
|Contaminantes| $NO$, $NO_2$, $O_3$, $PM_{10}$|
|Fecha Inicio| 01 Enero 2015 |
|Fecha Final | 31 Diciembre 2020 |
| Site Type | traffic |
|site area | urban |

In [4]:
# fechas de inicio y final de toma de datos
start_dt <- ymd_hms("2013-01-01 00:00:00")

In [5]:
# obtener datos de CA de España. Salen los códigos de las estaciones
# de Calidad de aire (941)
spain.sites <- get_saq_sites() %>%
    filter(country == "spain",
           site %in% sites.100mil$"Código.estación",
           site_type == "traffic",
           site_area == "urban",
           date_start <= start_dt,
           #date_end >= end_dt,
           ) %>%
    select(site, site_name, latitude, longitude, elevation, 
           country, site_type, site_area, date_start, date_end)

Agrupamos en un solo data.frame toda la informacion relevante de las estaciones de estudio

In [6]:
sites.geo <- sites.100mil %>%
    select("Municipio", "Población", 
           "Estación.tráfico", "Código.estación") %>%
    rename(site = "Código.estación",)

sitesAQ <- merge(x = spain.sites, y = sites.geo, by = "site", all.x = TRUE)
head(sitesAQ)

ERROR: Error: Can't subset columns that don't exist.
[31mx[39m Column `Poblaci<U+00F3>n` doesn't exist.


In [8]:
removed.sites <- (nrow(sites.100mil)-1) - nrow(sitesAQ)

print(paste("Se han eliminado", removed.sites, "estaciones"))
print(paste("Quedan", nrow(spain.sites), "estaciones para el estudio"))

[1] "Se han eliminado 11 estaciones"
[1] "Quedan 67 estaciones para el estudio"


### Guardar Datos en csv <a id="saveAQ"></a>

```R
write.csv(sitesAQ, 
          "data/Curation/AirQuality/sitesAQ.csv", row.names=FALSE)
```